In [2]:
import pandas as pd
import json
from preprocessing import preprocess
from functools import reduce


In [3]:
df_train = pd.read_csv('datasets/train_radiomics_hipocamp.csv')
df_test = pd.read_csv('datasets/test_radiomics_hipocamp.csv')

In [4]:
# df_train = preprocess(df_train,mode="all")

In [11]:
x_columns = list(map(lambda x:(x.split("_")),df_train.columns))
x_columns

[['ID'],
 ['Image'],
 ['Mask'],
 ['diagnostics', 'Versions', 'PyRadiomics'],
 ['diagnostics', 'Versions', 'Numpy'],
 ['diagnostics', 'Versions', 'SimpleITK'],
 ['diagnostics', 'Versions', 'PyWavelet'],
 ['diagnostics', 'Versions', 'Python'],
 ['diagnostics', 'Configuration', 'Settings'],
 ['diagnostics', 'Configuration', 'EnabledImageTypes'],
 ['diagnostics', 'Image-original', 'Hash'],
 ['diagnostics', 'Image-original', 'Dimensionality'],
 ['diagnostics', 'Image-original', 'Spacing'],
 ['diagnostics', 'Image-original', 'Size'],
 ['diagnostics', 'Image-original', 'Mean'],
 ['diagnostics', 'Image-original', 'Minimum'],
 ['diagnostics', 'Image-original', 'Maximum'],
 ['diagnostics', 'Mask-original', 'Hash'],
 ['diagnostics', 'Mask-original', 'Spacing'],
 ['diagnostics', 'Mask-original', 'Size'],
 ['diagnostics', 'Mask-original', 'BoundingBox'],
 ['diagnostics', 'Mask-original', 'VoxelNum'],
 ['diagnostics', 'Mask-original', 'VolumeNum'],
 ['diagnostics', 'Mask-original', 'CenterOfMassInde

In [12]:

def recursive_merge(dict1, dict2):
    """
    Recursively merge two dictionaries. Combines sub-dictionaries where both have the same key.
    """
    result = dict1.copy()  # Start with a shallow copy of the first dictionary
    for key, value in dict2.items():
        if key in result and isinstance(result[key], dict) and isinstance(value, dict):
            result[key] = recursive_merge(result[key], value)
        else:
            result[key] = value
    return result


def gen_trie(l):
    if l == []:
        return dict()
    else:
        head = l[0]
        tail = l[1:]

        if head == []:
            return gen_trie(tail)
        else:
            h_head = head[0]
            h_tail = head[1:]
            rec = gen_trie(tail)
            return recursive_merge({h_head:gen_trie([h_tail])} ,rec)
gen_trie(x_columns)

{'ID': {},
 'Image': {},
 'Mask': {},
 'diagnostics': {'Versions': {'PyRadiomics': {},
   'Numpy': {},
   'SimpleITK': {},
   'PyWavelet': {},
   'Python': {}},
  'Configuration': {'Settings': {}, 'EnabledImageTypes': {}},
  'Image-original': {'Hash': {},
   'Dimensionality': {},
   'Spacing': {},
   'Size': {},
   'Mean': {},
   'Minimum': {},
   'Maximum': {}},
  'Mask-original': {'Hash': {},
   'Spacing': {},
   'Size': {},
   'BoundingBox': {},
   'VoxelNum': {},
   'VolumeNum': {},
   'CenterOfMassIndex': {},
   'CenterOfMass': {}}},
 'original': {'shape': {'Elongation': {},
   'Flatness': {},
   'LeastAxisLength': {},
   'MajorAxisLength': {},
   'Maximum2DDiameterColumn': {},
   'Maximum2DDiameterRow': {},
   'Maximum2DDiameterSlice': {},
   'Maximum3DDiameter': {},
   'MeshVolume': {},
   'MinorAxisLength': {},
   'Sphericity': {},
   'SurfaceArea': {},
   'SurfaceVolumeRatio': {},
   'VoxelVolume': {}},
  'firstorder': {'10Percentile': {},
   '90Percentile': {},
   'Energy': {

In [ ]:
with open("cols_trie.json","+w") as f:
    json.dump(cols_trie,f,indent=2,sort_keys=True)
def tree_size(tree):
    res = 0
    for v in tree.values():
        res += 1
        if v != {}:
            res += tree_size(v)
    return res

def n_leaves(tree):
    res = 0
    for v in tree.values():
        if v != {}:
            res += tree_size(v)
        else:
            res += 1
    return res

In [ ]:
list(cols_trie.keys())
res = []
for k,v in cols_trie.items():
    res = res + [(k,n_leaves(v))]
res